In [66]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [67]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 25

In [68]:
CLASS_NAMES = ['Bacterial Spot','Early Blight','Healthy','Late Blight', 'Septoria Leaf Spot', 'Yellow Leaf Curl Virus']

In [69]:
train_path= "/kaggle/input/plant-village-dataset-updated/Tomato/Train"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    shuffle=True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 11108 files belonging to 6 classes.


In [70]:
test_path= "/kaggle/input/plant-village-dataset-updated/Tomato/Test"

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path, 
    shuffle=False,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 280 files belonging to 6 classes.


In [71]:
print(len(train_ds))
print(len(test_ds))

348
9


In [72]:
train_ds = train_ds.cache().shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

In [73]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.Rescaling(1./255),
])

In [74]:
num_classes = 6 # Number of classes

def preprocess(image, label):
    label = tf.cast(label, tf.int32)
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_ds = train_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Replace Flatten with GlobalAveragePooling2D
    GlobalAveragePooling2D(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()



Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 110,534 (431.77 KB)

 Trainable params: 110,534 (431.77 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [77]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data generator with the desired augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the images to [0, 1]
    rotation_range=15,  # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # Randomly shift images vertically (fraction of total height)
    shear_range=0.1,  # Randomly shear images
    zoom_range=0.1,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in any pixels lost during augmentation
)

# Define your validation data generator (usually without augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

In [78]:
train_generator = train_datagen.flow_from_directory(
    train_path,  # This is the source directory for training images
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  # All images will be resized to 224x224
    batch_size=32,
    class_mode='categorical'
)

Found 11108 images belonging to 6 classes.


In [79]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode='categorical'
)

Found 280 images belonging to 6 classes.


In [81]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint directory and file name
checkpoint_path = '/kaggle/working/cnn_model.weights.h5'

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,  # Save only the best model based on validation loss
    save_freq='epoch'     # Save after every epoch
)


In [82]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    verbose=1,
    callbacks=[cp_callback]
)

Epoch 1/25
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 635ms/step - accuracy: 0.3693 - loss: 1.4894
Epoch 1: val_loss improved from inf to 1.21901, saving model to /kaggle/working/cnn_model.weights.h5
347/347 ━━━━━━━━━━━━━━━━━━━━ 232s 644ms/step - accuracy: 0.3696 - loss: 1.4887 - val_accuracy: 0.5430 - val_loss: 1.2190
Epoch 2/25
  1/347 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.6562 - loss: 0.9379
Epoch 2: val_loss did not improve from 1.21901
347/347 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6562 - loss: 0.9379 - val_accuracy: 0.4583 - val_loss: 1.3537
Epoch 3/25
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.6453 - loss: 0.9287
Epoch 3: val_loss did not improve from 1.21901
347/347 ━━━━━━━━━━━━━━━━━━━━ 155s 437ms/step - accuracy: 0.6453 - loss: 0.9285 - val_accuracy: 0.5703 - val_loss: 1.3378
Epoch 4/25
  1/347 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.7188 - loss: 0.7198
Epoch 4: val_loss did not improve from 1.21901
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 84us/step - acc

In [65]:
model.save('tomato_disease_model.keras')